## Mejora del dataset de aeropuertos

#### Google Places nos da muchos más puntos de los reales cuando buscamos por tipo 'airport', así que haremos geoqueries a mongo

Importamos librerías

In [1]:
from pymongo import MongoClient
import pandas as pd
from src.gameslib import (toGeoJSON)
import requests
import json

Importamos dataset de aeropuertos descargado de https://datahub.io/core/airport-codes

In [2]:
# $ mongoimport --db companies --collection airports --jsonArray --drop inputs/airport-codes_json.json

Conectamos con la BD

In [3]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

Eliminamos todos los que no sean de tipo 'large_airport'

In [4]:

query_air = {"$and":[{"type":{"$not":{"$eq":"small_airport"}}},{"type":{"$not":{"$eq":"closed"}}},{"type":{"$not":{"$eq":"heliport"}}},{"type":{"$not":{"$eq":"seaplane_base"}}},{"type":{"$not":{"$eq":"balloonport"}}},{"type":{"$not":{"$eq":"medium_airport"}}}]}
aeropuertos = list(db["airports"].find(query_air))
dfa = pd.DataFrame(aeropuertos)

Le aplicamos transformaciones a la columna de coordenadas para obtener el formato correcto del punto para MongoDB

In [5]:
temp = dfa["coordinates"].str.split(", ", n = 1, expand = True)
dfa["longitud"]= temp[0].apply(lambda x:float(x))
dfa["latitud"]= temp[1].apply(lambda x:float(x))
dfa.drop(columns =["_id","coordinates"], inplace = True)
dfa["location"] = dfa[["latitud","longitud"]].apply(lambda x:toGeoJSON(x.longitud,x.latitud), axis=1)

# Guardamos fichero
dfa.to_json("./inputs/cleaned_airports.json", orient="records", force_ascii=False)

Importamos a mongodb una vez añadido el punto para geoqueries

In [6]:
# $ mongoimport --db companies --collection cleanairports --jsonArray --drop inputs/cleaned_airports.json

#### Importante crear en Mongo Compass un índice a la colección (2dsphere)